In [1]:
import logging
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from fonduer import Meta
from fonduer.candidates import CandidateExtractor, MentionExtractor, MentionNgrams
from fonduer.candidates.models import (
    Candidate,
    Mention,
    candidate_subclass,
    mention_subclass,
)

from fonduer.features import Featurizer
from fonduer.learning import SparseLogisticRegression
from fonduer.parser.models import Document, Figure, Paragraph, Section, Sentence
from fonduer.supervision import Labeler
from metal import analysis
from metal.label_model import LabelModel

from hack.transistors.transistor_lfs import (
    TRUE,
    ce_v_max_lfs,
    polarity_lfs,
    stg_temp_max_lfs,
    stg_temp_min_lfs,
)
from hack.transistors.transistor_matchers import get_matcher
from hack.transistors.transistor_spaces import (
    MentionNgramsPart,
    MentionNgramsTemp,
    MentionNgramsVolt,
)
from hack.transistors.transistor_throttlers import (
    ce_v_max_filter,
    polarity_filter,
    stg_temp_filter,
)
from hack.transistors.transistor_utils import (
    Score,
    entity_level_scores,
    load_transistor_labels,
)
from hack.utils import parse_dataset

In [2]:
# Use the first set of GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Configure logging for Hack
logging.basicConfig(
    format="[%(asctime)s][%(levelname)s] %(name)s:%(lineno)s - %(message)s",
    level=logging.INFO,
    handlers=[
        logging.FileHandler(
            os.path.join(os.path.abspath(''), f"transistors.log")
        ),
        logging.StreamHandler(),
    ],
)
logger = logging.getLogger(__name__)

In [3]:
def parsing(session, first_time=True, parallel=1, max_docs=float("inf")):
#     dirname = os.path.dirname(__file__)
    dirname = os.path.abspath('')
    logger.debug(f"Starting parsing...")
    docs, train_docs, dev_docs, test_docs = parse_dataset(
        session, dirname, first_time=first_time, parallel=parallel, max_docs=max_docs
    )
    logger.debug(f"Done")

    logger.info(f"# of train Documents: {len(train_docs)}")
    logger.info(f"# of dev Documents: {len(dev_docs)}")
    logger.info(f"# of test Documents: {len(test_docs)}")

    logger.info(f"Documents: {session.query(Document).count()}")
    logger.info(f"Sections: {session.query(Section).count()}")
    logger.info(f"Paragraphs: {session.query(Paragraph).count()}")
    logger.info(f"Sentences: {session.query(Sentence).count()}")
    logger.info(f"Figures: {session.query(Figure).count()}")

    return docs, train_docs, dev_docs, test_docs

In [4]:
def mention_extraction(
    session,
    docs,
    first_time=True,
    part=True,
    stg_temp_min=True,
    stg_temp_max=True,
    polarity=True,
    ce_v_max=True,
    parallel=1,
):
    Part = mention_subclass("Part")
    part_matcher = get_matcher("part")
    part_ngrams = MentionNgramsPart(parts_by_doc=None, n_max=3)

    StgTempMin = mention_subclass("StgTempMin")
    stg_temp_min_matcher = get_matcher("stg_temp_min")
    stg_temp_min_ngrams = MentionNgramsTemp(n_max=2)

    StgTempMax = mention_subclass("StgTempMax")
    stg_temp_max_matcher = get_matcher("stg_temp_max")
    stg_temp_max_ngrams = MentionNgramsTemp(n_max=2)

    Polarity = mention_subclass("Polarity")
    polarity_matcher = get_matcher("polarity")
    polarity_ngrams = MentionNgrams(n_max=1)

    CeVMax = mention_subclass("CeVMax")
    ce_v_max_matcher = get_matcher("ce_v_max")
    ce_v_max_ngrams = MentionNgramsVolt(n_max=1)

    mentions = []
    ngrams = []
    matchers = []

    # Only do those that are incrementally enabled
    if part:
        mentions.append(Part)
        ngrams.append(part_ngrams)
        matchers.append(part_matcher)

    if stg_temp_min:
        mentions.append(StgTempMin)
        ngrams.append(stg_temp_min_ngrams)
        matchers.append(stg_temp_min_matcher)

    if stg_temp_max:
        mentions.append(StgTempMax)
        ngrams.append(stg_temp_max_ngrams)
        matchers.append(stg_temp_max_matcher)

    if polarity:
        mentions.append(Polarity)
        ngrams.append(polarity_ngrams)
        matchers.append(polarity_matcher)

    if ce_v_max:
        mentions.append(CeVMax)
        ngrams.append(ce_v_max_ngrams)
        matchers.append(ce_v_max_matcher)

    mention_extractor = MentionExtractor(session, mentions, ngrams, matchers)

    if first_time:
        mention_extractor.apply(docs, parallelism=parallel)

    logger.info(f"Total Mentions: {session.query(Mention).count()}")
    logger.info(f"Total Part: {session.query(Part).count()}")
    logger.info(f"Total StgTempMin: {session.query(StgTempMin).count()}")
    logger.info(f"Total StgTempMax: {session.query(StgTempMax).count()}")
    logger.info(f"Total Polarity: {session.query(Polarity).count()}")
    logger.info(f"Total CeVMax: {session.query(CeVMax).count()}")
    return Part, StgTempMin, StgTempMax, Polarity, CeVMax

In [5]:
def candidate_extraction(
    session,
    Part,
    StgTempMin,
    StgTempMax,
    Polarity,
    CeVMax,
    train_docs,
    dev_docs,
    test_docs,
    stg_temp_min=True,
    stg_temp_max=True,
    polarity=True,
    ce_v_max=True,
    first_time=True,
    parallel=1,
):
    PartStgTempMin = candidate_subclass("PartStgTempMin", [Part, StgTempMin])
    stg_temp_min_throttler = stg_temp_filter

    PartStgTempMax = candidate_subclass("PartStgTempMax", [Part, StgTempMax])
    stg_temp_max_throttler = stg_temp_filter

    PartPolarity = candidate_subclass("PartPolarity", [Part, Polarity])
    polarity_throttler = polarity_filter

    PartCeVMax = candidate_subclass("PartCeVMax", [Part, CeVMax])
    ce_v_max_throttler = ce_v_max_filter

    cands = []
    throttlers = []
    if stg_temp_min:
        cands.append(PartStgTempMin)
        throttlers.append(stg_temp_min_throttler)

    if stg_temp_max:
        cands.append(PartStgTempMax)
        throttlers.append(stg_temp_max_throttler)

    if polarity:
        cands.append(PartPolarity)
        throttlers.append(polarity_throttler)

    if ce_v_max:
        cands.append(PartCeVMax)
        throttlers.append(ce_v_max_throttler)

    candidate_extractor = CandidateExtractor(session, cands, throttlers=throttlers)

    if first_time:
        for i, docs in enumerate([train_docs, dev_docs, test_docs]):
            candidate_extractor.apply(docs, split=i, parallelism=parallel)
            num_cands = session.query(Candidate).filter(Candidate.split == i).count()
            logger.info(f"Candidates in split={i}: {num_cands}")

    train_cands = candidate_extractor.get_candidates(split=0)
    dev_cands = candidate_extractor.get_candidates(split=1)
    test_cands = candidate_extractor.get_candidates(split=2)

    logger.info(f"Total train candidate: {len(train_cands[0])}")
    logger.info(f"Total dev candidate: {len(dev_cands[0])}")
    logger.info(f"Total test candidate: {len(test_cands[0])}")

    return (
        PartStgTempMin,
        PartStgTempMax,
        PartPolarity,
        PartCeVMax,
        train_cands,
        dev_cands,
        test_cands,
    )

In [6]:
def featurization(
    session,
    train_cands,
    dev_cands,
    test_cands,
    PartStgTempMin,
    PartStgTempMax,
    PartPolarity,
    PartCeVMax,
    stg_temp_min=True,
    stg_temp_max=True,
    polarity=True,
    ce_v_max=True,
    first_time=True,
    parallel=1,
):
#     dirname = os.path.dirname(__file__)
    dirname = os.path.abspath('')
    cands = []
    if stg_temp_min:
        cands.append(PartStgTempMin)

    if stg_temp_max:
        cands.append(PartStgTempMax)

    if polarity:
        cands.append(PartPolarity)

    if ce_v_max:
        cands.append(PartCeVMax)

    featurizer = Featurizer(session, cands)
    if first_time:
        logger.info("Starting featurizer...")
        featurizer.apply(split=0, train=True, parallelism=parallel)
        featurizer.apply(split=1, parallelism=parallel)
        featurizer.apply(split=2, parallelism=parallel)
        logger.info("Done")

    logger.info("Getting feature matrices...")
    # Serialize feature matrices on first run
    if first_time:
        F_train = featurizer.get_feature_matrices(train_cands)
        F_dev = featurizer.get_feature_matrices(dev_cands)
        F_test = featurizer.get_feature_matrices(test_cands)
        pickle.dump(F_train, open(os.path.join(dirname, "F_train.pkl"), "wb"))
        pickle.dump(F_dev, open(os.path.join(dirname, "F_dev.pkl"), "wb"))
        pickle.dump(F_test, open(os.path.join(dirname, "F_test.pkl"), "wb"))
    else:
        F_train = pickle.load(open(os.path.join(dirname, "F_train.pkl"), "rb"))
        F_dev = pickle.load(open(os.path.join(dirname, "F_dev.pkl"), "rb"))
        F_test = pickle.load(open(os.path.join(dirname, "F_test.pkl"), "rb"))
    logger.info("Done.")

    for i, cand in enumerate(cands):
        logger.info(f"{cand} Train shape: {F_train[i].shape}")
        logger.info(f"{cand} Test shape: {F_test[i].shape}")
        logger.info(f"{cand} Dev shape: {F_dev[i].shape}")

    return F_train, F_dev, F_test

In [7]:
def load_labels(session, relation, cand, first_time=True):
    if first_time:
        logger.info(f"Loading gold labels for {relation.value}")
        load_transistor_labels(session, [cand], [relation.value], annotator_name="gold")

In [8]:
def labeling(session, cands, cand_classes, lfs, split=1, train=False, first_time=True, parallel=1):
    labeler = Labeler(session, cand_classes)
#     if relation == Relation.STG_TEMP_MIN:
#         lfs = stg_temp_min_lfs
#     elif relation == Relation.STG_TEMP_MAX:
#         lfs = stg_temp_max_lfs
#     elif relation == Relation.POLARITY:
#         lfs = polarity_lfs
#     elif relation == Relation.CE_V_MAX:
#         lfs = ce_v_max_lfs
#     else:
#         raise ValueError(f"Invalid Relation: {relation}")

#     lfs = ce_v_max_lfs
    
    if first_time:
        logger.info("Applying LFs...")
        labeler.apply(split=split, lfs=lfs, train=train, parallelism=parallel)
        logger.info("Done...")

    logger.info("Getting label matrices...")
    L_mat = labeler.get_label_matrices(cands)
    L_gold = labeler.get_gold_labels(cands, annotator="gold")
    logger.info("Done.")
    logger.info(f"L_mat shape: {L_mat[0].shape}")
    logger.info(f"L_gold shape: {L_gold[0].shape}")

#     if train:
#         try:
#             df = analysis.lf_summary(
#                 L_mat[0],
#                 lf_names=labeler.get_keys(),
#                 Y=L_gold[0].todense().reshape(-1).tolist()[0],
#             )
#             logger.info(f"\n{df.to_string()}")
#         except Exception:
#             import pdb

#             pdb.set_trace()

    return L_mat, L_gold

In [9]:
def generative_model(L_train, n_epochs=500, print_every=100):
    model = LabelModel(k=2)

    logger.info("Training generative model...")
    model.train_model(L_train, n_epochs=n_epochs, print_every=print_every)
    logger.info("Done.")

    marginals = model.predict_proba(L_train)
#     plt.hist(marginals[:, TRUE - 1], bins=20)
#     plt.savefig(f"{relation.value}_marginals.pdf")
    return marginals

In [10]:
def discriminative_model(train_cands, F_train, marginals, n_epochs=50, lr=0.001):
    disc_model = SparseLogisticRegression()

    logger.info("Training discriminative model...")
    disc_model.train(
        (train_cands, F_train),
        marginals,
        n_epochs=n_epochs,
        lr=lr,
        host_device="GPU",
    )
    logger.info("Done.")

    return disc_model

In [29]:
def scoring(relation, disc_model, test_cands, test_docs, F_test, parts_by_doc, num=100):
    logger.info("Calculating the best F1 score and threshold (b)...")

    # Iterate over a range of `b` values in order to find the b with the
    # highest F1 score. We are using cardinality==2. See fonduer/classifier.py.
    Y_prob = disc_model.marginals((test_cands, F_test))

    # Get prediction for a particular b, store the full tuple to output
    # (b, pref, rec, f1, TP, FP, FN)
    best_result = Score(0, 0, 0, [], [], [])
    best_b = 0
    for b in np.linspace(0, 1, num=num):
        try:
            test_score = np.array(
                [TRUE if p[TRUE - 1] > b else 3 - TRUE for p in Y_prob]
            )
            true_pred = [
                test_cands[_] for _ in np.nditer(np.where(test_score == TRUE))
            ]
            result = entity_level_scores(
                true_pred,
                attribute=relation,
                corpus=test_docs,
                parts_by_doc=parts_by_doc,
            )
            logger.info(f"b = {b}, f1 = {result.f1}")
            if result.f1 > best_result.f1:
                best_result = result
                best_b = b
        except Exception as e:
            logger.debug(f"{e}, skipping.")
            break

    logger.info("===================================================")
    logger.info(f"Scoring on Entity-Level Gold Data with b={best_b}")
    logger.info("===================================================")
    logger.info(f"Corpus Precision {best_result.prec:.3f}")
    logger.info(f"Corpus Recall    {best_result.rec:.3f}")
    logger.info(f"Corpus F1        {best_result.f1:.3f}")
    logger.info("---------------------------------------------------")
    logger.info(
        f"TP: {len(best_result.TP)} "
        f"| FP: {len(best_result.FP)} "
        f"| FN: {len(best_result.FN)}"
    )
    logger.info("===================================================\n")
    return best_result, best_b

In [12]:
# See https://docs.python.org/3/library/os.html#os.cpu_count
parallel = 16  # len(os.sched_getaffinity(0)) // 4
component = "transistors"
first_time = True
max_docs = 500
conn_string = f"postgresql:///{component}_all_relations"
logger.info(f"\n\n")
logger.info(f"=" * 80)
logger.info(f"Small transistors with parallel: {parallel}, max_docs: {max_docs}")

[2019-03-07 21:37:13,767][INFO] __main__:7 - 


[2019-03-07 21:37:13,769][INFO] __main__:8 - ================================================================================
[2019-03-07 21:37:13,771][INFO] __main__:9 - Small transistors with parallel: 16, max_docs: 500


In [13]:
session = Meta.init(conn_string).Session()
docs, train_docs, dev_docs, test_docs = parsing(
    session, first_time=True, parallel=parallel, max_docs=max_docs
)

[2019-03-07 21:37:13,916][INFO] fonduer.meta:86 - Connecting user:None to None:None/transistors_all_relations
[2019-03-07 21:37:13,921][INFO] fonduer.meta:110 - Initializing the storage schema
[2019-03-07 21:37:14,401][INFO] hack.utils.utils:41 - Parsing dev...
[2019-03-07 21:37:28,486][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 21:40:43,912][INFO] hack.utils.utils:41 - Parsing test...


[2019-03-07 21:40:53,659][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 21:43:04,249][INFO] hack.utils.utils:41 - Parsing train...


[2019-03-07 21:44:19,789][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 22:00:06,349][INFO] __main__:10 - # of train Documents: 500
[2019-03-07 22:00:06,352][INFO] __main__:11 - # of dev Documents: 120
[2019-03-07 22:00:06,353][INFO] __main__:12 - # of test Documents: 75
[2019-03-07 22:00:06,364][INFO] __main__:14 - Documents: 695
[2019-03-07 22:00:06,374][INFO] __main__:15 - Sections: 695
[2019-03-07 22:00:07,367][INFO] __main__:16 - Paragraphs: 392426
[2019-03-07 22:00:08,550][INFO] __main__:17 - Sentences: 406080
[2019-03-07 22:00:09,715][INFO] __main__:18 - Figures: 19463


In [14]:
Part, StgTempMin, StgTempMax, Polarity, CeVMax = mention_extraction(
    session, docs, first_time=True, parallel=parallel
)

[2019-03-07 22:00:10,214][INFO] fonduer.candidates.mentions:460 - Clearing table: part
[2019-03-07 22:00:10,259][INFO] fonduer.candidates.mentions:460 - Clearing table: stg_temp_min
[2019-03-07 22:00:10,264][INFO] fonduer.candidates.mentions:460 - Clearing table: stg_temp_max
[2019-03-07 22:00:10,269][INFO] fonduer.candidates.mentions:460 - Clearing table: polarity
[2019-03-07 22:00:10,274][INFO] fonduer.candidates.mentions:460 - Clearing table: ce_v_max
[2019-03-07 22:00:10,277][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 22:03:05,270][INFO] __main__:67 - Total Mentions: 26609
[2019-03-07 22:03:05,289][INFO] __main__:68 - Total Part: 13308
[2019-03-07 22:03:05,300][INFO] __main__:69 - Total StgTempMin: 2025
[2019-03-07 22:03:05,313][INFO] __main__:70 - Total StgTempMax: 7691
[2019-03-07 22:03:05,323][INFO] __main__:71 - Total Polarity: 2624
[2019-03-07 22:03:05,332][INFO] __main__:72 - Total CeVMax: 961


In [15]:
(
    PartStgTempMin,
    PartStgTempMax,
    PartPolarity,
    PartCeVMax,
    train_cands,
    dev_cands,
    test_cands,
) = candidate_extraction(
    session,
    Part,
    StgTempMin,
    StgTempMax,
    Polarity,
    CeVMax,
    train_docs,
    dev_docs,
    test_docs,
    first_time=True,
    parallel=parallel,
)

[2019-03-07 22:03:05,446][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_min (split 0)
[2019-03-07 22:03:05,510][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_max (split 0)
[2019-03-07 22:03:05,513][INFO] fonduer.candidates.candidates:125 - Clearing table part_polarity (split 0)
[2019-03-07 22:03:05,516][INFO] fonduer.candidates.candidates:125 - Clearing table part_ce_v_max (split 0)
[2019-03-07 22:03:05,519][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 22:10:38,116][INFO] __main__:54 - Candidates in split=0: 212493
[2019-03-07 22:10:38,117][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_min (split 1)
[2019-03-07 22:10:38,122][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_max (split 1)
[2019-03-07 22:10:38,130][INFO] fonduer.candidates.candidates:125 - Clearing table part_polarity (split 1)
[2019-03-07 22:10:38,136][INFO] fonduer.candidates.candidates:125 - Clearing table part_ce_v_max (split 1)
[2019-03-07 22:10:38,141][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 22:11:46,118][INFO] __main__:54 - Candidates in split=1: 53942
[2019-03-07 22:11:46,120][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_min (split 2)
[2019-03-07 22:11:46,125][INFO] fonduer.candidates.candidates:125 - Clearing table part_stg_temp_max (split 2)
[2019-03-07 22:11:46,130][INFO] fonduer.candidates.candidates:125 - Clearing table part_polarity (split 2)
[2019-03-07 22:11:46,136][INFO] fonduer.candidates.candidates:125 - Clearing table part_ce_v_max (split 2)
[2019-03-07 22:11:46,140][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 22:11:58,493][INFO] __main__:54 - Candidates in split=2: 5210


[2019-03-07 22:12:07,740][INFO] __main__:60 - Total train candidate: 10829
[2019-03-07 22:12:07,742][INFO] __main__:61 - Total dev candidate: 7272
[2019-03-07 22:12:07,743][INFO] __main__:62 - Total test candidate: 566


In [19]:
from hack.transistors.transistor_utils import (
    Score,
    entity_level_scores,
    entity_to_candidates,
)

from pprint import pformat

In [20]:
import pickle
pickle_file = 'data/parts_by_doc_new.pkl'
with open(pickle_file, 'rb') as f:
    parts_by_doc = pickle.load(f)

In [21]:
# First, check total recall

for i, name in enumerate(['stg_temp_min', 'stg_temp_max', 'polarity', 'ce_v_max']):
    logger.info(name)
    result = entity_level_scores(dev_cands[i], corpus=dev_docs, attribute=name, parts_by_doc=parts_by_doc)
    logger.info(f"Gain Total Dev Recall: {result.rec:.3f}")
    # logger.info(f"\n{pformat(result.FN)}")
    result = entity_level_scores(test_cands[i], corpus=test_docs, attribute=name, parts_by_doc=parts_by_doc)
    logger.info(f"Gain Total Test Recall: {result.rec:.3f}")
    # logger.info(f"\n{pformat(result.FN)}")


[2019-03-07 22:15:09,290][INFO] __main__:4 - stg_temp_min


KeyError: 'BC550'

In [22]:
F_train, F_dev, F_test = featurization(
    session,
    train_cands,
    dev_cands,
    test_cands,
    PartStgTempMin,
    PartStgTempMax,
    PartPolarity,
    PartCeVMax,
    first_time=first_time,
    parallel=parallel,
)

[2019-03-07 22:16:11,959][INFO] __main__:34 - Starting featurizer...
[2019-03-07 22:16:21,269][INFO] fonduer.features.featurizer:190 - Clearing Features (split 0)
[2019-03-07 22:16:21,279][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 23:43:04,366][INFO] fonduer.features.featurizer:190 - Clearing Features (split 1)
[2019-03-07 23:43:05,018][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 23:45:09,093][INFO] fonduer.features.featurizer:190 - Clearing Features (split 2)
[2019-03-07 23:45:09,133][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-07 23:45:47,999][INFO] __main__:38 - Done
[2019-03-07 23:45:48,002][INFO] __main__:40 - Getting feature matrices...
[2019-03-07 23:56:08,506][INFO] __main__:53 - Done.
[2019-03-07 23:56:08,509][INFO] __main__:56 - <class 'fonduer.candidates.models.candidate.PartStgTempMin'> Train shape: (10829, 55127)
[2019-03-07 23:56:08,511][INFO] __main__:57 - <class 'fonduer.candidates.models.candidate.PartStgTempMin'> Test shape: (566, 55127)
[2019-03-07 23:56:08,512][INFO] __main__:58 - <class 'fonduer.candidates.models.candidate.PartStgTempMin'> Dev shape: (7272, 55127)
[2019-03-07 23:56:08,513][INFO] __main__:56 - <class 'fonduer.candidates.models.candidate.PartStgTempMax'> Train shape: (110738, 55127)
[2019-03-07 23:56:08,514][INFO] __main__:57 - <class 'fonduer.candidates.models.candidate.PartStgTempMax'> Test shape: (1848, 55127)
[2019-03-07 23:56:08,516][INFO] __main__:58 - <class 'fonduer.candidates.models.candidate.PartStgTempMax'> Dev shape: (27473, 55127)
[2019-03-07 23:56:08,5

In [25]:
# load_labels(session, relation, PartCeVMax, first_time=first_time)
logger.info("Labeling training data...")
L_train, L_gold_train = labeling(
    session, train_cands, \
    [PartStgTempMin, PartStgTempMax, PartPolarity, PartCeVMax], \
    [stg_temp_min_lfs, stg_temp_max_lfs, polarity_lfs, ce_v_max_lfs], \
    split=0, train=True, parallel=parallel, first_time=True
)
logger.info("Done.")

[2019-03-08 00:58:09,036][INFO] __main__:2 - Labeling training data...
[2019-03-08 00:58:09,039][INFO] __main__:17 - Applying LFs...
[2019-03-08 00:58:15,015][INFO] fonduer.supervision.labeler:219 - Clearing Labels (split 0)
[2019-03-08 00:58:16,688][INFO] fonduer.utils.udf:57 - Running UDF...


[2019-03-08 01:32:21,876][INFO] __main__:19 - Done...
[2019-03-08 01:32:21,893][INFO] __main__:21 - Getting label matrices...
[2019-03-08 01:40:27,461][INFO] __main__:24 - Done.
[2019-03-08 01:40:27,463][INFO] __main__:25 - L_mat shape: (10829, 44)
[2019-03-08 01:40:27,465][INFO] __main__:26 - L_gold shape: (10829, 1)
[2019-03-08 01:40:27,467][INFO] __main__:9 - Done.


In [31]:
marginals = generative_model(L_train[0])
disc_model1 = discriminative_model(train_cands[0], F_train[0], marginals, n_epochs=100)

relation = 'stg_temp_min'
best_result, best_b = scoring(
    relation, disc_model1, test_cands[0], test_docs, F_test[0], parts_by_doc, num=100
)

[2019-03-08 02:15:50,564][INFO] __main__:4 - Training generative model...


Computing O...
Estimating \mu...
[E:0]	Train Loss: 3.392
[E:100]	Train Loss: 0.003
[E:200]	Train Loss: 0.003
[E:300]	Train Loss: 0.003
[E:400]	Train Loss: 0.003


[2019-03-08 02:15:53,068][INFO] __main__:6 - Done.
[2019-03-08 02:15:53,095][INFO] __main__:4 - Training discriminative model...
[2019-03-08 02:15:53,099][INFO] fonduer.learning.classifier:142 - Loading default parameters for Sparse Logistic Regression
[2019-03-08 02:15:53,130][INFO] fonduer.learning.classifier:193 - Using GPU...
[2019-03-08 02:15:53,132][INFO] fonduer.learning.classifier:195 - Settings: {'n_epochs': 100, 'lr': 0.001, 'batch_size': 256, 'shuffle': True, 'seed': 1234, 'host_device': 'GPU', 'bias': False, 'input_dim': 55128}
[2019-03-08 02:15:53,144][INFO] fonduer.learning.classifier:213 - [SparseLogisticRegression] Training model
[2019-03-08 02:15:53,146][INFO] fonduer.learning.classifier:215 - [SparseLogisticRegression] n_train=7389 #epochs=100 batch size=256


[E:499]	Train Loss: 0.003
Finished Training


[2019-03-08 02:15:54,406][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 1 (1.26s)	Average loss=0.223570
[2019-03-08 02:15:58,994][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 5 (5.85s)	Average loss=0.061675
[2019-03-08 02:16:04,189][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 10 (11.04s)	Average loss=0.056004
[2019-03-08 02:16:09,478][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 15 (16.33s)	Average loss=0.054554
[2019-03-08 02:16:14,778][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 20 (21.63s)	Average loss=0.054032
[2019-03-08 02:16:19,877][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 25 (26.73s)	Average loss=0.053799
[2019-03-08 02:16:25,413][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 30 (32.27s)	Average loss=0.053634
[2019-03-08 02:16:30,928][INFO] fonduer.learning.classifier:262 - [Sparse

...

[2019-03-08 02:17:58,154][INFO] __main__:26 - b = 0.9494949494949496, f1 = 0.7235772357723577


[2019-03-08 02:17:58,291][INFO] __main__:26 - b = 0.9595959595959597, f1 = 0.7235772357723577


[2019-03-08 02:17:58,428][INFO] __main__:26 - b = 0.9696969696969697, f1 = 0.7235772357723577


[2019-03-08 02:17:58,582][INFO] __main__:26 - b = 0.9797979797979799, f1 = 0.7235772357723577


[2019-03-08 02:17:58,716][INFO] __main__:26 - b = 0.98989898989899, f1 = 0.71900826446281
[2019-03-08 02:17:58,720][INFO] __main__:34 - ===================================================
[2019-03-08 02:17:58,721][INFO] __main__:35 - Scoring on Entity-Level Gold Data with b=0.9191919191919192
[2019-03-08 02:17:58,721][INFO] __main__:36 - ===================================================
[2019-03-08 02:17:58,722][INFO] __main__:37 - Corpus Precision 0.957
[2019-03-08 02:17:58,723][INFO] __main__:38 - Corpus Recall    0.582
[2019-03-08 02:17:58,723][INFO] __main__:39 - Corpus F1        0.724
[2019-03-08 02:17:58,724][INFO] __main__:40 - ---------------------------------------------------
[2019-03-08 02:17:58,725][INFO] __main__:42 - TP: 89 | FP: 4 | FN: 64
[2019-03-08 02:17:58,725][INFO] __main__:46 - ===================================================



In [32]:
marginals = generative_model(L_train[1])
disc_model1 = discriminative_model(train_cands[1], F_train[1], marginals, n_epochs=100)

relation = 'stg_temp_max'
best_result, best_b = scoring(
    relation, disc_model1, test_cands[1], test_docs, F_test[1], parts_by_doc, num=100
)

[2019-03-08 02:17:58,737][INFO] __main__:4 - Training generative model...


Computing O...
Estimating \mu...
[E:0]	Train Loss: 0.833
[E:100]	Train Loss: 0.043
[E:200]	Train Loss: 0.042
[E:300]	Train Loss: 0.042
[E:400]	Train Loss: 0.042


[2019-03-08 02:18:01,416][INFO] __main__:6 - Done.


[E:499]	Train Loss: 0.042
Finished Training


[2019-03-08 02:18:01,689][INFO] __main__:4 - Training discriminative model...
[2019-03-08 02:18:01,696][INFO] fonduer.learning.classifier:142 - Loading default parameters for Sparse Logistic Regression
[2019-03-08 02:18:04,227][INFO] fonduer.learning.classifier:193 - Using GPU...
[2019-03-08 02:18:04,229][INFO] fonduer.learning.classifier:195 - Settings: {'n_epochs': 100, 'lr': 0.001, 'batch_size': 256, 'shuffle': True, 'seed': 1234, 'host_device': 'GPU', 'bias': False, 'input_dim': 55128}
[2019-03-08 02:18:04,240][INFO] fonduer.learning.classifier:213 - [SparseLogisticRegression] Training model
[2019-03-08 02:18:04,241][INFO] fonduer.learning.classifier:215 - [SparseLogisticRegression] n_train=110738 #epochs=100 batch size=256
[2019-03-08 02:18:18,279][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 1 (14.04s)	Average loss=0.483407
[2019-03-08 02:19:12,453][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 5 (68.21s)	Average loss=0.47617

...

[2019-03-08 02:40:56,598][INFO] __main__:26 - b = 0.9494949494949496, f1 = 0.75098814229249


[2019-03-08 02:40:56,744][INFO] __main__:26 - b = 0.9595959595959597, f1 = 0.7410358565737052


[2019-03-08 02:40:56,890][INFO] __main__:26 - b = 0.9696969696969697, f1 = 0.736


[2019-03-08 02:40:57,034][INFO] __main__:26 - b = 0.9797979797979799, f1 = 0.7309236947791165


[2019-03-08 02:40:57,178][INFO] __main__:26 - b = 0.98989898989899, f1 = 0.7154471544715448
[2019-03-08 02:40:57,188][INFO] __main__:34 - ===================================================
[2019-03-08 02:40:57,189][INFO] __main__:35 - Scoring on Entity-Level Gold Data with b=0.9494949494949496
[2019-03-08 02:40:57,189][INFO] __main__:36 - ===================================================
[2019-03-08 02:40:57,190][INFO] __main__:37 - Corpus Precision 0.979
[2019-03-08 02:40:57,191][INFO] __main__:38 - Corpus Recall    0.609
[2019-03-08 02:40:57,191][INFO] __main__:39 - Corpus F1        0.751
[2019-03-08 02:40:57,192][INFO] __main__:40 - ---------------------------------------------------
[2019-03-08 02:40:57,193][INFO] __main__:42 - TP: 95 | FP: 2 | FN: 61
[2019-03-08 02:40:57,194][INFO] __main__:46 - ===================================================



In [33]:
marginals = generative_model(L_train[2])
disc_model1 = discriminative_model(train_cands[2], F_train[2], marginals, n_epochs=100)

relation = 'polarity'
best_result, best_b = scoring(
    relation, disc_model1, test_cands[2], test_docs, F_test[2], parts_by_doc, num=100
)

[2019-03-08 02:40:57,203][INFO] __main__:4 - Training generative model...


Computing O...
Estimating \mu...
[E:0]	Train Loss: 0.505
[E:100]	Train Loss: 0.054
[E:200]	Train Loss: 0.045
[E:300]	Train Loss: 0.045
[E:400]	Train Loss: 0.045


[2019-03-08 02:40:59,712][INFO] __main__:6 - Done.
[2019-03-08 02:40:59,816][INFO] __main__:4 - Training discriminative model...
[2019-03-08 02:40:59,821][INFO] fonduer.learning.classifier:142 - Loading default parameters for Sparse Logistic Regression


[E:499]	Train Loss: 0.045
Finished Training


[2019-03-08 02:41:00,030][INFO] fonduer.learning.classifier:193 - Using GPU...
[2019-03-08 02:41:00,031][INFO] fonduer.learning.classifier:195 - Settings: {'n_epochs': 100, 'lr': 0.001, 'batch_size': 256, 'shuffle': True, 'seed': 1234, 'host_device': 'GPU', 'bias': False, 'input_dim': 55128}
[2019-03-08 02:41:00,043][INFO] fonduer.learning.classifier:213 - [SparseLogisticRegression] Training model
[2019-03-08 02:41:00,044][INFO] fonduer.learning.classifier:215 - [SparseLogisticRegression] n_train=68199 #epochs=100 batch size=256
[2019-03-08 02:41:09,090][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 1 (9.05s)	Average loss=0.219603
[2019-03-08 02:41:37,532][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 5 (37.49s)	Average loss=0.186457
[2019-03-08 02:42:12,537][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 10 (72.49s)	Average loss=0.184800
[2019-03-08 02:42:47,965][INFO] fonduer.learning.classifier:262 - [Sp

...

[2019-03-08 02:53:02,153][INFO] __main__:26 - b = 0.9393939393939394, f1 = 0.15294117647058825


[2019-03-08 02:53:02,309][INFO] __main__:26 - b = 0.9494949494949496, f1 = 0.11976047904191617


[2019-03-08 02:53:02,465][INFO] __main__:26 - b = 0.9595959595959597, f1 = 0.10909090909090909


[2019-03-08 02:53:02,623][INFO] __main__:26 - b = 0.9696969696969697, f1 = 0.012903225806451615


[2019-03-08 02:53:02,778][INFO] __main__:26 - b = 0.9797979797979799, f1 = 0.012903225806451615
[2019-03-08 02:53:02,790][INFO] __main__:34 - ===================================================
[2019-03-08 02:53:02,791][INFO] __main__:35 - Scoring on Entity-Level Gold Data with b=0.0
[2019-03-08 02:53:02,792][INFO] __main__:36 - ===================================================
[2019-03-08 02:53:02,793][INFO] __main__:37 - Corpus Precision 0.855
[2019-03-08 02:53:02,794][INFO] __main__:38 - Corpus Recall    0.922
[2019-03-08 02:53:02,794][INFO] __main__:39 - Corpus F1        0.887
[2019-03-08 02:53:02,795][INFO] __main__:40 - ---------------------------------------------------
[2019-03-08 02:53:02,796][INFO] __main__:42 - TP: 141 | FP: 24 | FN: 12
[2019-03-08 02:53:02,797][INFO] __main__:46 - ===================================================



In [28]:
marginals = generative_model(L_train[3])
disc_model1 = discriminative_model(train_cands[3], F_train[3], marginals, n_epochs=100)

[2019-03-08 02:10:31,873][INFO] __main__:4 - Training generative model...


Computing O...
Estimating \mu...
[E:0]	Train Loss: 27.843
[E:100]	Train Loss: 0.097
[E:200]	Train Loss: 0.092
[E:300]	Train Loss: 0.091
[E:400]	Train Loss: 0.091


[2019-03-08 02:10:34,322][INFO] __main__:6 - Done.
[2019-03-08 02:10:34,363][INFO] __main__:4 - Training discriminative model...
[2019-03-08 02:10:34,366][INFO] fonduer.learning.classifier:142 - Loading default parameters for Sparse Logistic Regression
[2019-03-08 02:10:34,431][INFO] fonduer.learning.classifier:193 - Using GPU...
[2019-03-08 02:10:34,432][INFO] fonduer.learning.classifier:195 - Settings: {'n_epochs': 100, 'lr': 0.001, 'batch_size': 256, 'shuffle': True, 'seed': 1234, 'host_device': 'GPU', 'bias': False, 'input_dim': 55128}
[2019-03-08 02:10:34,449][INFO] fonduer.learning.classifier:213 - [SparseLogisticRegression] Training model
[2019-03-08 02:10:34,450][INFO] fonduer.learning.classifier:215 - [SparseLogisticRegression] n_train=17506 #epochs=100 batch size=256


[E:499]	Train Loss: 0.091
Finished Training


[2019-03-08 02:10:36,489][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 1 (2.04s)	Average loss=0.385586
[2019-03-08 02:10:43,719][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 5 (9.27s)	Average loss=0.301332
[2019-03-08 02:10:52,574][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 10 (18.12s)	Average loss=0.297307
[2019-03-08 02:11:01,369][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 15 (26.92s)	Average loss=0.295298
[2019-03-08 02:11:10,872][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 20 (36.42s)	Average loss=0.295058
[2019-03-08 02:11:20,253][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 25 (45.80s)	Average loss=0.294835
[2019-03-08 02:11:30,416][INFO] fonduer.learning.classifier:262 - [SparseLogisticRegression] Epoch 30 (55.97s)	Average loss=0.294493
[2019-03-08 02:11:39,480][INFO] fonduer.learning.classifier:262 - [Sparse

In [30]:
relation = 'ce_v_max'
best_result, best_b = scoring(
    relation, disc_model1, test_cands[3], test_docs, F_test[3], parts_by_doc, num=100
)

[2019-03-08 02:14:55,355][INFO] __main__:2 - Calculating the best F1 score and threshold (b)...


[2019-03-08 02:15:00,396][INFO] __main__:26 - b = 0.0, f1 = 0.8178694158075602


[2019-03-08 02:15:00,576][INFO] __main__:26 - b = 0.010101010101010102, f1 = 0.8178694158075602


[2019-03-08 02:15:00,811][INFO] __main__:26 - b = 0.020202020202020204, f1 = 0.8178694158075602


[2019-03-08 02:15:00,991][INFO] __main__:26 - b = 0.030303030303030304, f1 = 0.8178694158075602


...

[2019-03-08 02:15:16,846][INFO] __main__:26 - b = 0.9494949494949496, f1 = 0.8178694158075602


[2019-03-08 02:15:17,016][INFO] __main__:26 - b = 0.9595959595959597, f1 = 0.8178694158075602


[2019-03-08 02:15:17,188][INFO] __main__:26 - b = 0.9696969696969697, f1 = 0.8178694158075602


[2019-03-08 02:15:17,358][INFO] __main__:26 - b = 0.9797979797979799, f1 = 0.8055555555555556


[2019-03-08 02:15:17,525][INFO] __main__:26 - b = 0.98989898989899, f1 = 0.7045454545454545
[2019-03-08 02:15:17,530][INFO] __main__:34 - ===================================================
[2019-03-08 02:15:17,531][INFO] __main__:35 - Scoring on Entity-Level Gold Data with b=0.0
[2019-03-08 02:15:17,532][INFO] __main__:36 - ===================================================
[2019-03-08 02:15:17,532][INFO] __main__:37 - Corpus Precision 0.875
[2019-03-08 02:15:17,533][INFO] __main__:38 - Corpus Recall    0.768
[2019-03-08 02:15:17,534][INFO] __main__:39 - Corpus F1        0.818
[2019-03-08 02:15:17,535][INFO] __main__:40 - ---------------------------------------------------
[2019-03-08 02:15:17,535][INFO] __main__:42 - TP: 119 | FP: 17 | FN: 36
[2019-03-08 02:15:17,536][INFO] __main__:46 - ===================================================

